In [1]:
gROOT->GetVersion()

(const char *) "6.26/02"


In [2]:
%jsroot on

In [3]:
double P_K = 3;
double m_K = 0.498;
double m_pi = 0.1396;
double m_mu = 0.1057;
double m_numu = 0.0002;
double angle = TMath::Pi()/8;

In [4]:
TLorentzVector K_L{P_K*TMath::Cos(angle), 0, P_K*TMath::Sin(angle), TMath::Sqrt(P_K*P_K+m_K*m_K)};
TGenPhaseSpace event;
double masses[] = {m_pi, m_mu, m_numu};
event.SetDecay(K_L, 3, masses);

In [5]:
ROOT::RDataFrame df(100000);

In [6]:
auto phasespace = df
    .Define("weight", "event.Generate()")
    .Define("pi", "*event.GetDecay(0)")
    .Define("l", "*event.GetDecay(1)")
    .Define("nu", "*event.GetDecay(2)");

In [7]:
auto phasespace_with_vars = phasespace
.Define("piP", "sqrt(pi.Px()*pi.Px()+pi.Py()*pi.Py()+pi.Pz()*pi.Pz())")
.Define("lP", "sqrt(l.Px()*l.Px()+l.Py()*l.Py()+l.Pz()*l.Pz())")
.Define("candidate", "pi+l")
.Define("candidateP", "sqrt(candidate.Px()*candidate.Px()+candidate.Py()*candidate.Py()+candidate.Pz()*candidate.Pz())")
.Define("IP0x", "abs(candidate.Px()/candidateP)*35")
.Define("IP0y", "abs(candidate.Py()/candidateP)*35");

In [8]:
ROOT::RDF::RSnapshotOptions opts;
opts.fLazy = true;
phasespace_with_vars.Snapshot("phasespace", "K_L_phasespace_augmented.root", "", opts);

In [9]:
auto phasespace_with_cuts = phasespace_with_vars
.Filter("piP > 1 && lP > 1", "Product momentum")
.Filter("pi.Pz() > 0 && l.Pz() > 0", "Product direction")
.Filter("IP0y < 7.5", "Approximate fiducial/IP0 Y")
.Filter("IP0x < 5", "Approximate fiducial/IP0 X");

In [10]:
auto report = phasespace_with_cuts.Report();

In [11]:
report->Print();

Product momentum: pass=24581      all=100000     -- eff=24.58 % cumulative eff=24.58 %
Product direction: pass=24581      all=24581      -- eff=100.00 % cumulative eff=24.58 %
Approximate fiducial/IP0 Y: pass=24581      all=24581      -- eff=100.00 % cumulative eff=24.58 %
Approximate fiducial/IP0 X: pass=0          all=24581      -- eff=0.00 % cumulative eff=0.00 %
